In [1]:
!pip uninstall -y google-generativeai
!pip install -U google-genai


Found existing installation: google-generativeai 0.8.6
Uninstalling google-generativeai-0.8.6:
  Successfully uninstalled google-generativeai-0.8.6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.1/719.1 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 25.3 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.43.0
    Uninstalling google-auth-2.43.0:
      Successfully uninstalled google-auth-2.43.0
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.55.0
    Uninstalling google-genai-1.55.0:
      Successfully uninstalled google-genai-1.55.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 

In [3]:
df=pd.read_csv("labeled_full.csv")
df['label'].value_counts()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,count
label,
neutral,681
against,179
agreed,27


In [1]:
!pip install -U setfit pandas scikit-learn

In [4]:
import pandas as pd
import torch
from setfit import SetFitModel, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

# 1. VERIFY GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Working on: {device}")

# 2. PREPARE DATA
# Assuming 'df' is your dataframe with 887 entries
# If you need to load it: df = pd.read_csv('your_file.csv')

# Map labels to integers
label_map = {"agreed": 0, "neutral": 1, "against": 2}
df['label_int'] = df['label'].map(label_map)

# Merge columns for rich context
df['text'] = (
    "Title: " + df['title'].fillna('') +
    " | Description: " + df['description'].fillna('') +
    " | Keywords: " + df['keywords'].fillna('')
)

# 3. FIX IMBALANCE (Oversampling)
# We target around 400 samples per class so the model isn't biased toward Neutral
df_neutral = df[df.label_int == 1]
df_against = df[df.label_int == 2]
df_agreed = df[df.label_int == 0]

df_against_upsampled = resample(df_against, replace=True, n_samples=400, random_state=42)
df_agreed_upsampled = resample(df_agreed, replace=True, n_samples=400, random_state=42)
df_neutral_sampled = resample(df_neutral, replace=False, n_samples=400, random_state=42) # match others

df_balanced = pd.concat([df_neutral_sampled, df_against_upsampled, df_agreed_upsampled])

# 4. SPLIT
train_df, test_df = train_test_split(
    df_balanced,
    test_size=0.2,
    stratify=df_balanced['label_int'],
    random_state=42
)

train_ds = Dataset.from_pandas(train_df[['text', 'label_int']])
test_ds = Dataset.from_pandas(test_df[['text', 'label_int']])

# 5. LOAD MODEL
# Paraphrase-mpnet is the "Gold Standard" for this size
model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
    labels=["agreed", "neutral", "against"]
)
model.to(device) # Move to T4 GPU

# 6. TRAINING ARGUMENTS
# We set num_iterations=10 to keep it fast but accurate
args = TrainingArguments(
    batch_size=16,
    num_epochs=1,           # SetFit only needs 1 epoch for the contrastive phase
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    num_iterations=10       # This prevents the 119,000 steps error!
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    column_mapping={"text": "text", "label_int": "label"}
)

# 7. TRAIN
print("Starting training on GPU...")
trainer.train()

# 8. EVALUATE
metrics = trainer.evaluate()
print(f"Accuracy: {metrics['accuracy']:.4f}")

# 9. TEST IT
sample_texts = [
    "AI is revolutionizing the classroom and making teachers' lives easier.",
    "The decline in language courses is a tragedy for modern education.",
    "New report released on the statistics of university enrollment."
]
preds = model.predict(sample_texts)

for text, pred in zip(sample_texts, preds):
    print(f"Text: {text}\nPrediction: {pred}\n")

# 10. SAVE MODEL
model.save_pretrained("sentiment_stance_model")
print("Model saved successfully!")

Working on: cuda


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())
***** Running evaluation *****
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDat

/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())


Accuracy: 0.9250
Text: AI is revolutionizing the classroom and making teachers' lives easier.
Prediction: neutral

Text: The decline in language courses is a tragedy for modern education.
Prediction: against

Text: New report released on the statistics of university enrollment.
Prediction: neutral



/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Model saved successfully!


In [5]:
import shutil
from google.colab import files

# Zip the model folder
shutil.make_archive("stance_model_v1", 'zip', "sentiment_stance_model")

# Download it
files.download("stance_model_v1.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# Pick 5 random rows from your original dataframe
samples = df.sample(5)

for i, row in samples.iterrows():
    combined_text = f"Title: {row['title']} | Desc: {row['description']}"
    prediction = model.predict([combined_text])
    print(f"TEXT: {row['title']}")
    print(f"PREDICTION: {prediction[0]}")
    print("-" * 30)

TEXT: Google in 2026: From foundational gains to future expectations
PREDICTION: agreed
------------------------------
TEXT: Job killer
PREDICTION: against
------------------------------
TEXT: The Slop of Things to Come
PREDICTION: against
------------------------------


/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datet

TEXT: AI wonât cure disease but gold, silver & natureâs medicine can secure your future
PREDICTION: against
------------------------------
TEXT: The 2026 Reckoning: Americaâs collapse is now unstoppable
PREDICTION: against
------------------------------


/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=ut